<table>
  <tr>
    <th style="background-color:#ffe694;">
     
<font size=6> Manipuler un réseau de neurones en Python</font>
      </th>
  </tr>
</table>

## Dans ce chapitre  nous expliquons comment créer, entraîner et tester un réseau de neurones stanadard (feedforward et totalement connecté) en Python. Pour ce faire nous avons besoin du module <font color='blue'>keras</font> qui s'appuie sur le module <font color='blue'>tensorflow</font>, mais aussi du module <font color='blue'>sklearn</font>.  L'ensemble de données utilisé dans est *Iris*.  L'objectif est de prédire la classe *Species*.

In [11]:
import pandas as pn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

## Préparation des données : dans le cas d'un problème de multiclassification à m classes, on transforme les modalités de la classe en m-uplets ayant une seule valeur 1 et toutes les autres valeurs à 0. C'est la représentation one-hot que nous codons "à la main" ici mais pour laquelle Python fournit une fonction pré-définie que nous introduisons ailleurs.



In [12]:
# Encodage des classes
def encodeClass(s_class):
    if (s_class=='Iris-setosa'):
        return [1,0,0]
    elif (s_class=='Iris-versicolor'):
        return [0,1,0]
    else:
        return [0,0,1]

## Lecture des données et extraction des caractéristiques qui nous serviront dans la définition du RN.

In [13]:
#Lecture des données
myData=pn.read_csv('data\\iris.csv', sep=',')

#Extraction de quelques caractéristiques de l'ensemble de données
nbColumns = len(myData.columns)
classes = myData['species'].unique().tolist()
nbClasses = len(classes)

print('Nombre de neurones de la couche d\'entrée = nombre de variables indépendantes = ',nbColumns-1)
print('Nombre de neurones de la couche de sortie = nombre de modalités de la classe = ',nbClasses)

Nombre de neurones de la couche d'entrée = nombre de variables indépendantes =  4
Nombre de neurones de la couche de sortie = nombre de modalités de la classe =  3


## Préparation des ensembles d'apprentissage et des test.

In [38]:
#Nous avons de changer de structures de données en passant d'une dataframe à deux tableaux numpy.
X=myData.values[:,:nbColumns-1]
X=X.astype('float64')
Y=myData.values[:,nbColumns-1]

#Encodage des classes
encoded_Y = np.array([encodeClass(y) for y in list(Y)])

#Création des ensembles d'apprentissage et de test
X_train, X_test, Y_train, Y_test = train_test_split( X, encoded_Y, test_size = 0.4, random_state = 100)


## Création du RNA: création d'un réseau 'vierge' puis ajout des couches une par une.

In [39]:
#Création d'un RNA vierge
nn = Sequential()

## Ajout des couches une par une. Le module keras met à notre disposition une fonction pour chaque type de couches. Ici, nous utilisons exclusivement la fonction *dense* correspondant aux couches du RN stanadrd. Pour ajouter une couche, on précise son nombre de neurones et sa fonction d'activation, et pour la première couche cachée le nombre de ses entrées (càd le nombre de neurones de la couche d'entrée).

In [40]:
#Ajout des couches
nn.add(Dense(5, input_dim=nbColumns-1, activation='sigmoid'))
nn.add(Dense(nbClasses, activation='softmax'))

## Affichage d'un "résumé" du RN. Ce résumé se présente sous la forme d'un tableau dans lequel chaque ligne correspond à une couche du réseau. La dernière colonne du tableau contient le nombre de paramètres (poids et biais) de chaque couche. On vérifie bien que pour chaque couche ce nombre est égal à n1*(n2+1), où n1 est le nombre de neurones de la couche et n2 celui de la couche précédente.

In [41]:
#Affichage d'un "résumé" du RNA
nn.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 25        
                                                                 
 dense_7 (Dense)             (None, 3)                 18        
                                                                 
Total params: 43 (172.00 Byte)
Trainable params: 43 (172.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Avant d'entraîner notre RN, nous complétons ses caractéristiques en définissant notamment la fonction de *perte* (*loss* function, ici la categorical_crossentropy car c'est un pb de multiclassification) et la métrique utilisée pour mesurer sa performance, ici la précision (*accuracy*).

In [42]:
nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entraînement du RN. En plus de l'ensemble d'apprentissage, on précise le nombre d'itérations (epochs) et le nombre d'exemples utilisés pour chaque étape de l'apprentissage.

In [43]:
nn.fit(X_train, Y_train, epochs=50, batch_size=10)

Epoch 1/50
9/9 [==============================] - 1s 1ms/step - loss: 1.4623 - accuracy: 0.3889
Epoch 2/50
9/9 [==============================] - 0s 2ms/step - loss: 1.4253 - accuracy: 0.3889
Epoch 3/50
9/9 [==============================] - 0s 4ms/step - loss: 1.3876 - accuracy: 0.3889
Epoch 4/50
9/9 [==============================] - 0s 1ms/step - loss: 1.3536 - accuracy: 0.3889
Epoch 5/50
9/9 [==============================] - 0s 2ms/step - loss: 1.3166 - accuracy: 0.3889
Epoch 6/50
9/9 [==============================] - 0s 858us/step - loss: 1.2877 - accuracy: 0.3889
Epoch 7/50
9/9 [==============================] - 0s 2ms/step - loss: 1.2591 - accuracy: 0.3889
Epoch 8/50
9/9 [==============================] - 0s 1ms/step - loss: 1.2305 - accuracy: 0.3889
Epoch 9/50
9/9 [==============================] - 0s 2ms/step - loss: 1.2088 - accuracy: 0.3889
Epoch 10/50
9/9 [==============================] - 0s 2ms/step - loss: 1.1893 - accuracy: 0.3889
Epoch 11/50
9/9 [====================

## Test du RN : Calcul et affichage de l'accuracy.

In [44]:
score = nn.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.8666666746139526


## Pour plus d'information, on construit la matrice de confusion.

In [45]:
Y_pred = nn.predict(X_test)
Y_pred_1 = Y_pred.argmax(axis=1)
Y_test_1 = Y_test.argmax(axis=1)
confusion = confusion_matrix(Y_pred_1, Y_test_1)
print(confusion) 

2/2 [==============================] - 0s 8ms/step
[[22  0  0]
 [ 0 15  8]
 [ 0  0 15]]
